In [1]:
import cv2
import numpy as np

In [2]:
image=cv2.imread("13.jpeg")
H,w=image.shape[:2]
n_w=416
aspect_r=n_w/w
new_H=int(H*aspect_r)
img_resize=cv2.resize(image,(n_w,new_H),interpolation=cv2.INTER_AREA)


In [3]:
gray=cv2.cvtColor(img_resize,cv2.COLOR_BGR2GRAY)
mblur=cv2.medianBlur(gray,11,2)
#erode=cv2.erode(blur,(3,3),5)
#kernel1=cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
#opening=cv2.morphologyEx(gray,cv2.MORPH_OPEN,kernel1)
Blur=cv2.GaussianBlur(gray,(3,3),0)
T,inv_thresh=cv2.threshold(mblur,0,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU )
canny=cv2.Canny(mblur,100,255)
erode=cv2.erode(inv_thresh,(7,7),5)
cv2.imshow('Image',inv_thresh)
cv2.imshow("Gray",gray)
#cv2.imshow('Canny',canny)
cv2.waitKey(0)

-1

# Finding and drawing contours
So up until this point in the course we have been able to apply methods like thresholding (Module 1.9) and edge detection (Module 1.10) to detect the outlines and structures of objects in images.

However, now that we have the outlines and structures of the objects in images, the big question becomes: How do we find and access these outlines?

The answer: contours.

I’ll say this many times before this lesson is over, but contours are an invaluable tool to have in your tool belt. Being able to leverage simple contour properties enables you to solve complicated problems with ease.

Finding and Drawing Contours
Before we can utilize contour properties to identify X’s and O’s on a tic-tac-toe board or identify Tetris blocks, we first need to understand how to find contours in an image and draw them.

As I mentioned in the introduction to this lesson, contours are simply the outlines of an object in an image. If the image is simple enough, we might be able to get away with using the grayscale image as an input.

But for more complicated images, we must first find the object by using methods such as edge detection or thresholding — we are simply seeking a binary image where white pixels correspond to objects in an image and black pixels as the background. There are many ways to obtain a binary image like this, but the most used methods are edge detection and thresholding.

Key takeaway: For better accuracy you’ll normally want to utilize a binary image rather than a grayscale image. We’ll use both binary and grayscale images in this lesson. But once we start to get to some of the more advanced contour topics, you’ll notice that we switch from grayscale to binary images to improve our contour accuracy.

In [16]:
import imutils
contours=cv2.findContours(inv_thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#contours=cv2.findContours(gray.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
grab_cnts=imutils.grab_contours(contours)
clone=img_resize.copy()
#cv2.drawContours(clone,grab_cnts,-1,(0,0,0),2)
#print('Found {} contours'.format(len(grab_cnts)))
cv2.imshow("Contours",clone)
#cv2.imwrite('Clone_pest.jpg',clone)
cv2.waitKey(0)

-1

We make a call to cv2.findContours  on Line 20, passing in our grayscale image. The cv2.findContours  function is destructive to the input image (meaning that it manipulates it) so if you intend on using your input image again, be sure to clone it using the copy()  method prior to passing it into cv2.findContours .

We’ll instruct cv2.findContours  to return a list of all contours in the image by passing in the cv2.RETR_LIST  flag. This flag will ensure that all contours are returned. Other methods exist, such as returning only the external most contours, which we’ll explore later in this article.

Finally, we pass in the cv2.CHAIN_APPROX_SIMPLE  flag. If we did not specify this flag and instead used cv2.CHAIN_APPROX_NONE , we would be storing every single (x, y)-coordinate along the contour. In general, this not advisable. It’s substantially slower and takes up significantly more memory. By compressing our horizontal, vertical, and diagonal segments into only end-points we are able to reduce memory consumption significantly without any substantial loss in contour accuracy. You can read more about the compression of contours in the
# drawContours

The third parameter is the index of the contour inside the cnts  list that we want to draw. If we wanted to draw only the first contour, we could pass in a value of 0. If we wanted to draw only the second contour, we would supply a value of 1. Passing in a value of -1 for this argument instructs the cv2.drawContours  function to draw all contours in the cnts  list. Personally, I like to always supply a value of -1 and just supply the single contour manually. Doing this is slightly more Pythonic and easier to read — I’ll make this point clear later in this lesson.

Finally, the last two arguments to the cv2.drawContours  function is the color of the contour (in this case green), and the thickness of the contour line (2 pixels).

Consider what would happen if I wanted to draw the first two contours and nothing else. I would either have to manually make two calls to cv2.findContours  or to use a for  loop and supply the contour index value i.

Simply put, that’s not very Pythonic when all I need to do is supply Python array slices:

Drawing the first two contours
cv2.drawContours
## (clone, cnts[:2], -1, (0, 255, 0), 2)

# Centroid/Center of Mass
The “centroid” or “center of mass” is the center (x, y)-coordinate of an object in an image. This (x, y)-coordinate is actually calculated based on the image moments, which are based on the weighted average of the (x, y)-coordinates/pixel intensity along the contour.
## Bounding Boxes
A bounding box is exactly what it sounds like — an upright rectangle that “bounds” and “contains” the entire contoured region of the image. However, it does not consider the rotation of the shape, so you’ll want to keep that in mind.

A bounding box consists of four components: the starting x-coordinate of the box, then the starting y-coordinate of the box, followed by the width and height of the box.

In [23]:
for c in grab_cnts:
    (x,y,w,h)=cv2.boundingRect(c)
    cv2.rectangle(clone,(x,y),(x+w,y+h),(0,0,255),2)
    cv2.putText(clone,"Fruitfly",(x-1,y+1),1,cv2.FONT_HERSHEY_PLAIN,(0,0,255),1)
cv2.imshow('Clone',clone)
cv2.waitKey(0)

-1

In [27]:
for c in grab_cnts:
    box = cv2.minAreaRect(c)
    (x,y),(w,h),(theta)=box
    x=int(x)
    y=int(y)
    box = np.int0(cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box))
    cv2.drawContours(clone, [box], -1, (0, 255, 0), 2)
    cv2.putText(clone,"Fruitfly",(x-1,y+1),1,cv2.FONT_HERSHEY_PLAIN,(0,0,255),1)
cv2.imshow("Clone",clone)
#cv2.imwrite("Image_processing.jpg",clone)
cv2.waitKey(0)

-1

The cv2.minAreaRect  (Line 75) function takes our contour and returns a tuple with 3 values. The first value of the tuple is the starting (x, y)-coordinates of the rotated bounding box. The second value is the width and height of the bounding box. And the final value is our \theta, or angle of rotation of the shape.

However, since we want to draw a rotated bounding box rather than a standard bounding box we won’t be able to leverage the cv2.rectangle  function. Instead, we’ll pass the output of cv2.minAreaRect  to the cv2.boxPoints  (Line 76) function (cv2.cv.BoxPoints  for OpenCV 2.4) which converts the (x, y)-coordinates, width and height, and angle of rotation into a set of coordinates points.

In essence, the cv2.minAreaRect  function just gives us another contour, which we then draw on our image on Line 77.

Below you can see the output of computing the rotated bounding boxes:

Figure 8: Computing rotated bounding boxes rather than standard ones.
Figure 4: Computing rotated bounding boxes rather than standard ones.
In general, you’ll want to use standard bounding boxes when you want to crop a shape from an image (Module 1.4.5). And you’ll want to use rotated bounding boxes when you are utilizing masks to extract regions from an image.

In [33]:
for c in grab_cnts:
    (x,y),radius=cv2.minEnclosingCircle(c)
    cv2.circle(clone,(int(x),int(y)),int(radius),(255,0,0),4)
    cv2.putText(clone,"Circle",(int(x),int(y)),1,cv2.FONT_HERSHEY_PLAIN,(255,0,0),1)
cv2.imshow("Circle",clone)
cv2.waitKey(0)

-1

In [34]:
#Fitting an Ellipse
#Fitting an ellipse to a contour is much like fitting a rotated rectangle to a contour.

#Under the hood, OpenCV is computing the rotated rectangle of the contour. And then it’s taking the rotated rectangle and computing an ellipse to fit in the rotated region:

#contour_properties_1.py
# loop over the contours
for c in grab_cnts:
	# to fit an ellipse, our contour must have at least 5 points
	if len(c) >= 5:
		# fit an ellipse to the contour
		ellipse = cv2.fitEllipse(c)
		cv2.ellipse(clone, ellipse, (0, 255, 0), 2)
# show the output image
cv2.imshow("Ellipses", clone)
cv2.waitKey(0)

-1